##Установка и импорт пакетов
###Подготовка



In [1]:
!pip install SentencePiece
!pip install transformers
!rm github-typo-corpus.v1.0.0.jsonl
!wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
!gzip -d github-typo-corpus.v1.0.0.jsonl.gz 
!pip install corus
!rm lenta-ru-news.csv.bz2
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2
!pip install wiki-ru-wordnet
!pip install razdel
!pip install pytorch_lightning

     |████████████████████████████████| 1.2MB 8.0MB/s 
     |████████████████████████████████| 2.3MB 8.0MB/s 
     |████████████████████████████████| 3.3MB 40.4MB/s 
     |████████████████████████████████| 901kB 51.1MB/s 
rm: cannot remove 'github-typo-corpus.v1.0.0.jsonl': No such file or directory
--2021-06-04 17:39:26--  https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
Resolving github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)... 52.217.88.212
Connecting to github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)|52.217.88.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43769081 (42M) [application/x-gzip]
Saving to: ‘github-typo-corpus.v1.0.0.jsonl.gz’

github-typo-corpus. 100%[===================>]  41.74M  27.1MB/s    in 1.5s    

2021-06-04 17:39:28 (27.1 MB/s) - ‘github-typo-corpus.v1.0.0.jsonl.gz’ saved [43769081/43769081]

     |████████████████████████████████| 92kB 6.2M

In [2]:

# Загрузка необходимых библиотек и модулей

import zipfile # Библиотека для работы с zip архивами
import os      # Библиотека для работы с фаловой системой 
import time    # Библиотека для работы со временем

from google.colab import drive # Модуль для работы с Google Dis
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

SRC_TEXT = './drive/MyDrive/ML/src_text50000.txt'
TGT_TEXT = './drive/MyDrive/ML/tgt_text50000.txt'

In [3]:
import json
import pandas as pd
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import corus
from corus import load_lenta2
from sklearn.model_selection import train_test_split
from razdel import tokenize as tokenize_razdel
from razdel import sentenize as sentenize_razdel
import random
import string
import requests
from datetime import datetime
from google.colab import files
from wiki_ru_wordnet import WikiWordnet

In [5]:


with open('./github-typo-corpus.v1.0.0.jsonl', 'r') as json_file:
    json_list = list(json_file)
rus_texts = []
langs = set()
for json_str in json_list:
    result = json.loads(json_str)
    lang = result.get("edits")[0].get("src").get('lang')
    langs.add(lang)
    if lang == 'rus':
      rus_texts.append(result);

In [6]:
src_texts = [ text.get('edits')[0].get('src').get('text') for text in rus_texts]
tgt_texts = [ text.get('edits')[0].get('tgt').get('text') for text in rus_texts]
del rus_texts

In [7]:
df = pd.DataFrame(list(zip(src_texts, tgt_texts)), columns=['src', 'tgt'])

In [4]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50",src_lang='ru_RU', tgt_lang='ru_RU')

In [9]:
tokenized_srcs = [tokenizer(text) for text in df['src']]
tokenized_tgts = [tokenizer(text) for text in df['tgt']]
del df


##Уже не стоит запускать

In [ ]:
tokenized_srcs

In [ ]:
tokenizer.decode(tokenized_srcs[10]['input_ids'], skip_special_tokens=True)

'* **HBibTeX-Styles:** Подборка русских стилевых пакетов BibTeX под UTF-8 '

In [ ]:
path = './lenta-ru-news.csv.bz2'
records = load_lenta2(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/1914/09/16/hungarnn/',
    title='1914. Русские войска вступили в\xa0пределы Венгрии  ',
    text='Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии

In [ ]:
records = list(records)
records[0]

LentaRecord(
    url='https://lenta.ru/news/1914/09/16/lermontov/',
    title='1914. Празднование столетия М.Ю. Лермонтова отложено ',
    text='Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного времени.  «Русский инвалид», 16 сентября 1914 года.',
    topic='Библиотека',
    tags='Первая мировая',
    date=datetime.datetime(1914, 9, 16, 0, 0)
)

## **Функции для зашумления текста и самозашумление**


In [ ]:
def new_or_missing_letters(sentence):
  #razd = list(tokenize_razdel(sentence))
  splt_sent = sentence.split()
  #for i in range(len(list(tokenize_razdel(sentence)))):
    #splt_sent.append(list(tokenize_razdel(sentence))[i].text)
  iter = len(splt_sent)//4
  vocab = ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']
  #print(splt_sent)
  while iter > 0:
    # Удаляем букву
    chosen_word = random.randint(0, len(splt_sent)-1)
    splt_word = list(splt_sent[chosen_word])
    #print(splt_word)
    del splt_word[random.randint(0, len(splt_word)-1)]
    #print(splt_word)
    if splt_word:
      splt_sent[chosen_word] = "".join(splt_word)
    

    # Добавляем букву
    chosen_word = random.randint(0, len(splt_sent)-1)
    splt_word = list(splt_sent[chosen_word])
    splt_word.insert(random.randint(0, len(splt_word)), vocab[random.randint(0, len(vocab)-1)])
    splt_sent[chosen_word] = "".join(splt_word)
    iter -= 1
  
  new_sent = " ".join(splt_sent)
  return new_sent

In [ ]:
def add_word(sentence):
  splt_sent = sentence.split()
  iter = len(splt_sent)//5
  while iter > 0:
    chosen_word = random.randint(0, len(splt_sent)-1)
    #print(chosen_word)
    splt_sent.insert(chosen_word, splt_sent[chosen_word])
    #print(splt_sent)
    iter -= 1
  return " ".join(splt_sent)



In [ ]:
def del_word(sentence):
  splt_sent = sentence.split()
  iter = len(splt_sent)//5
  while iter > 0:
    del splt_sent[random.randint(0, len(splt_sent)-1)]
    iter -= 1
  return " ".join(splt_sent)



In [ ]:

store = {}
def semantic_replace(sentence):

  split_sentence = sentence.split()
  indexes = list(range(len(split_sentence)))
  while indexes:
    chosen_index = random.choice(indexes);
    i = indexes.index(chosen_index)
    chosen = split_sentence[chosen_index]
    if chosen in store.keys():
      split_sentence[chosen_index] = store.get(chosen)
      break
    else:
      response = requests.get("https://dictionary.yandex.net/api/v1/dicservice.json/lookup?key=dict.1.1.20210521T075501Z.ea7bfce571f63eda.dcc29036a656b4f718687ccee5f52aae19902465&lang=ru-ru&text={}".format(chosen))
      if response.json().get('def'):

        syn = response.json().get('def')[0].get('tr')
        syn = syn[- 1]
        if 'syn' in syn:
          syn = syn.get('syn')
          syn = syn[-1].get('text')

        else:
          syn = syn.get('text')
        split_sentence[chosen_index] = syn
        store[chosen] = syn
        break
      else:
        del indexes[i]
  return " ".join(split_sentence)


In [ ]:


def semantic_replace_slow(sentence):

  split_sentence = sentence.split()
  indexes = list(range(len(split_sentence)))
  while indexes:
    chosen_index = random.choice(indexes);
    i = indexes.index(chosen_index)
    chosen = split_sentence[chosen_index]
    vocab = []
    wikiwordnet = WikiWordnet()
    
    synsets = wikiwordnet.get_synsets(chosen)

    for synset in synsets:
      for w in synset.get_words():
        w = w.lemma()
        if w != chosen:
          vocab.append(w)
    print(vocab, chosen)
    if vocab:
      split_sentence[chosen_index] = random.choice(vocab)
      break
    else:
      del indexes[i]
  return " ".join(split_sentence)



In [ ]:
nouns_vocab = []
with open('russian_nouns.txt', 'r') as file:
  nouns_vocab = file.readlines()

In [ ]:
def semantic_replace_offline(sentence):
  
  split_sentence = sentence.split()
  for iter in range(0,len(split_sentence), 8):
    chosen_index = random.randint(iter, len(split_sentence)-1)
    split_sentence[chosen_index] = random.choice(nouns_vocab)[: -1]
  return " ".join(split_sentence)
    


In [ ]:

def make_tgt_sentences(news):
  rec = list(sentenize_razdel(news))
  data_news = []
  #chosen_mistake = random.randint(0, 4)
  for s in rec:
    chosen_mistake = random.randint(1, 4)

    if chosen_mistake == 1:
      data_news.append(new_or_missing_letters(s.text))
    elif chosen_mistake == 2:
      data_news.append(add_word(s.text))
    elif chosen_mistake == 3:
      data_news.append(del_word(s.text))
    elif chosen_mistake == 4:
      data_news.append(semantic_replace_offline(s.text))
    #elif chosen_mistake == 0:
      #data_news.append(semantic_replace(s.text))

  #print(data_news)
  return data_news
#print('------------------')


In [ ]:
def create_src(path, start=0, end= 50000):
  with open(path, 'w') as file:
    for i in range(start, end):
  
      text = make_tgt_sentences(records[i].text)
      #print(len(text))
      for str in text:
        str += '\n'
        file.write(str)
      file.write('\n')



In [ ]:
def create_tgt (path, start=0, end= 50000):
  with open(path, 'w') as file:
    for i in range(start, end):
  
      text = records[i].text
      for _, _, str in sentenize_razdel(text):
        str += '\n'
        file.write(str)

      file.write('\n')
    


## Загрузка и токенизация

In [10]:
def download_dataset(path, skip_news_splitter = False):
  dataset = []
  lines = []
  line_counter = 0
  rec = []
  with open(path, 'r') as file:
    lines = file.readlines()

  for i, line in enumerate(lines):
    if line != '\n':
      if line.endswith('\n'):
        line = line[0: -1]
      if skip_news_splitter:
        dataset.append(line)
      else:
        rec.append(line)
      line_counter += 1
    else: 
      if not skip_news_splitter:
        dataset.append(rec)
        rec = []
  return dataset, line_counter

In [11]:
src_texts, size = download_dataset(SRC_TEXT, True)
print(size)
tgt_texts, size = download_dataset(TGT_TEXT, True)
print(size)


411085
411085


In [12]:
tokenized_srcs_news = [tokenizer(line) for line in src_texts]
tokenized_tgts_news = [tokenizer(line) for line in tgt_texts]
del src_texts
del tgt_texts
tokenized_srcs_news[:10]

[{'input_ids': [250021, 94211, 154381, 591, 158279, 1114, 4, 49, 44959, 591, 89, 72499, 124234, 6, 237037, 2958, 128297, 4, 128297, 4, 9159, 1219, 529, 85979, 25337, 2151, 49, 6365, 91734, 4485, 1006, 529, 10262, 10262, 84242, 1435, 5, 18288, 5, 1435, 5, 18288, 5, 16308, 800, 127224, 2946, 4700, 9, 695, 60115, 60115, 41310, 2699, 16, 102069, 2454, 102069, 2454, 77733, 39242, 49, 49, 207401, 109408, 244, 753, 1315, 89, 1085, 129, 129, 165578, 105, 4, 183, 48034, 652, 19508, 21218, 96126, 6, 743, 28650, 103, 245, 255, 5138, 6, 101191, 1270, 13686, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [250021, 94, 1447, 109042, 12635, 35, 7533, 526, 1339, 611, 67165, 41310, 2699, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1,

In [13]:
for line in tokenized_srcs_news:
  tokenized_srcs.append(line)
for line in tokenized_tgts_news:
  tokenized_tgts.append(line)
print(len(tokenized_srcs))
print(len(tokenized_tgts))
del tokenized_srcs_news
del tokenized_tgts_news

411869
411869


In [14]:
whole_dataset = list(zip(tokenized_srcs, tokenized_tgts))
del tokenized_srcs
del tokenized_tgts
whole_dataset[0]

({'input_ids': [250021, 589, 9868, 160319, 1281, 361, 43883, 35, 52513, 103, 518, 5434, 6171, 74326, 30236, 195396, 4406, 336, 29692, 227, 5, 101090, 67668, 13394, 3280, 16788, 4019, 59, 9868, 10758, 1584, 9473, 135457, 748, 9, 207184, 939, 1135, 4, 6, 214809, 6033, 10422, 3447, 15, 19408, 380, 100550, 49, 127335, 22373, 526, 8492, 1270, 57800, 36062, 16, 49, 26318, 151024, 89, 5, 1721, 60834, 13394, 3280, 63080, 129583, 126094, 33689, 135, 147380, 1455, 358, 7365, 6467, 211783, 16662, 5, 6, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [250021, 589, 9868, 160319, 1281, 361, 43883, 35, 52513, 103, 518, 5434, 6171, 74326, 30236, 195396, 4406, 336, 29692, 227, 5, 101090, 67668, 13394, 3280, 16788, 4019, 59, 9868, 10758, 1584, 9473, 135457, 748, 9, 207184,

In [15]:
random.shuffle(whole_dataset)
whole_dataset[0]

({'input_ids': [250021, 7762, 4801, 232501, 2715, 255, 71066, 227, 49, 3577, 1041, 80804, 29611, 589, 30943, 698, 4, 19724, 158957, 2744, 29725, 591, 9430, 4135, 197, 20871, 136305, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [250021, 7762, 232501, 2715, 255, 71066, 227, 49, 35343, 80804, 29611, 589, 30943, 698, 4, 19724, 158957, 183, 29725, 42393, 197, 20871, 136305, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [16]:
src, tgt = whole_dataset[100]
print(tokenizer.decode(src['input_ids'], skip_special_tokens=True))
print(tokenizer.decode(tgt['input_ids'], skip_special_tokens=True))


Напомним, что в среду Эдвардс и Берлин конструктивистка себя виновными в совершении незаконных операций по переводу миллиардов долларов из России за рубеж через гуманитарий Bank of каплюшка ольшняк
Напомним, что в среду Эдвардс и Берлин признали себя виновными в совершении незаконных операций по переводу миллиардов долларов из России за рубеж через счета Bank of New York.


In [17]:
train_dataset = whole_dataset[:150000]
val_dataset = whole_dataset[150000: 200000 ]
del whole_dataset

In [18]:
from torch.nn.utils.rnn import pad_sequence

In [19]:
def batch_constructor(ds_rows):
    input_ids_list = []
    decoder_input_ids_list = []
    attention_mask_list = []
    decoder_attention_mask_list = []
    for src, tgt in ds_rows:
        input_ids_list.append(torch.tensor(src['input_ids'], dtype=torch.long))
        decoder_input_ids_list.append(torch.tensor(tgt['input_ids'], dtype=torch.long))
        attention_mask_list.append(torch.tensor(src['attention_mask'], dtype=torch.long))
        decoder_attention_mask_list.append(torch.tensor(tgt['attention_mask'], dtype=torch.long))
    
    return {
        'input_ids': pad_sequence(input_ids_list, batch_first=True, padding_value=tokenizer.pad_token_id),
        'attention_mask': pad_sequence(attention_mask_list, batch_first=True, padding_value=0),
        'labels': pad_sequence(decoder_input_ids_list, batch_first=True, padding_value=tokenizer.pad_token_id),
        'decoder_attention_mask': pad_sequence(decoder_attention_mask_list, batch_first=True, padding_value=0)
    }

In [20]:
from transformers import PretrainedConfig

class Seq2SeqConfig(PretrainedConfig):
    
    model_type = "mbart"
    keys_to_ignore_at_inference = ["past_key_values"]

    def __init__(
        self,
        vocab_size=50265,
        max_position_embeddings=1024,
        encoder_layers=12,
        encoder_ffn_dim=4096,
        encoder_attention_heads=16,
        decoder_layers=12,
        decoder_ffn_dim=4096,
        decoder_attention_heads=16,
        encoder_layerdrop=0.0,
        decoder_layerdrop=0.0,
        use_cache=True,
        is_encoder_decoder=True,
        activation_function="gelu",
        d_model=1024,
        dropout=0.1,
        attention_dropout=0.0,
        activation_dropout=0.0,
        init_std=0.02,
        classifier_dropout=0.0,
        scale_embedding=False,
        gradient_checkpointing=False,
        pad_token_id=1,
        bos_token_id=0,
        eos_token_id=2,
        forced_eos_token_id=2,
        omit_encoder_input_embeddings=False,
        separate_input_vocab=False,
        separate_input_vocab_size=None,
        separate_input_padding_idx=0,
        position_embedding_type='learned',
        **kwargs
    ):
        super().__init__(
            pad_token_id=pad_token_id,
            bos_token_id=bos_token_id,
            eos_token_id=eos_token_id,
            is_encoder_decoder=is_encoder_decoder,
            forced_eos_token_id=forced_eos_token_id,
            **kwargs,
        )
        self.vocab_size = vocab_size
        self.max_position_embeddings = max_position_embeddings
        self.d_model = d_model
        self.encoder_ffn_dim = encoder_ffn_dim
        self.encoder_layers = encoder_layers
        self.encoder_attention_heads = encoder_attention_heads
        self.decoder_ffn_dim = decoder_ffn_dim
        self.decoder_layers = decoder_layers
        self.decoder_attention_heads = decoder_attention_heads
        self.dropout = dropout
        self.attention_dropout = attention_dropout
        self.activation_dropout = activation_dropout
        self.activation_function = activation_function
        self.init_std = init_std
        self.encoder_layerdrop = encoder_layerdrop
        self.decoder_layerdrop = decoder_layerdrop
        self.classifier_dropout = classifier_dropout
        self.use_cache = use_cache
        self.num_hidden_layers = encoder_layers
        self.gradient_checkpointing = gradient_checkpointing
        self.scale_embedding = scale_embedding  # scale factor will be sqrt(d_model) if True
        self.omit_encoder_input_embeddings = omit_encoder_input_embeddings

        # Добавим опцию для отдельного словаря перед Encoder.
        assert bool(separate_input_vocab) == bool(separate_input_vocab_size)
        self.separate_input_vocab = separate_input_vocab
        self.separate_input_vocab_size = separate_input_vocab_size
        self.separate_input_padding_idx = separate_input_padding_idx
        self.position_embedding_type = position_embedding_type
        #
        # if self.separate_input_vocab:
        #     assert not self.tie_word_embeddings

    @property
    def num_attention_heads(self) -> int:
        return self.encoder_attention_heads

    @property
    def hidden_size(self) -> int:
        return self.d_model

In [21]:
config = Seq2SeqConfig(
    omit_encoder_input_embeddings=True,
    separate_input_vocab_size=tokenizer.vocab_size,
    separate_input_padding_idx=tokenizer.pad_token_id,
    separate_input_vocab=True,
    vocab_size=tokenizer.vocab_size,
    d_model=256,
    max_length=20,
    encoder_layers=3,
    decoder_layers=3,

    use_cache=False,
    decoder_ffn_dim = 1024,
    encoder_ffn_dim = 1024,
)

In [22]:
from torch.utils.data import DataLoader

In [23]:
train_dataloder = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=batch_constructor)
val_dataloder = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=batch_constructor)


In [24]:
hparams = {
    'huggingface_config':config.__dict__,
    'learning_rate': 5e-4,
    'warmup_steps': 500,
}

In [25]:
import pytorch_lightning as pl
from transformers import MBartConfig
from transformers.modeling_outputs import Seq2SeqLMOutput
from transformers.models.mbart import MBartForConditionalGeneration
from transformers.optimization import AdamW, get_constant_schedule_with_warmup
import torch



class BartConditionalGeneration(pl.LightningModule):

    def __init__(self, hparams):
      super().__init__()
      config_class = MBartConfig
      config = config_class(**hparams['huggingface_config'])
      model_class = MBartForConditionalGeneration
      self.seq2seq = model_class(config)
      self.save_hyperparameters(hparams)


    def forward_with_labels(self, batch):
        
        assert isinstance(batch, dict)
        assert 'labels' in batch and batch['labels'] is not None
        
        
        outputs: Seq2SeqLMOutput = self.seq2seq(**batch, return_dict=True)
        
        return outputs

    def training_step(self, batch, batch_idx):
        
        outputs: Seq2SeqLMOutput = self.forward_with_labels(batch)
        
        self.log('step_loss', outputs.loss, prog_bar=True)
        return outputs.loss

    def validation_step(self, batch, batch_idx):
        outputs: Seq2SeqLMOutput = self.forward_with_labels(batch)
        self.log('val_step_loss', outputs.loss, prog_bar=True)
        return {
            'val_step_loss': outputs.loss.item()
        }

    def validation_epoch_end(self, outputs) -> None:
        avg_loss = sum(d['val_step_loss'] for d in outputs) / len(outputs)
        self.log('avg_val_loss', avg_loss, prog_bar=True)

    def configure_optimizers(self):
        
        optimizer = AdamW(params=self.parameters(), lr=self.hparams['learning_rate'])
        warmup_steps = self.hparams['warmup_steps']
        scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)

        return [optimizer], [{
            'scheduler': scheduler,
            'interval': 'step'
        }]

In [26]:
model = BartConditionalGeneration(hparams)


In [27]:
type(model.seq2seq)

transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration

In [28]:
type(model.seq2seq.config)

transformers.models.mbart.configuration_mbart.MBartConfig

In [29]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import pytorch_lightning as pl

In [30]:
batch_example = next(iter(train_dataloder))
for k,v in batch_example.items():
    print(k, v.size())

input_ids torch.Size([2, 23])
attention_mask torch.Size([2, 23])
labels torch.Size([2, 25])
decoder_attention_mask torch.Size([2, 25])


In [31]:
early_stop_callback = EarlyStopping(
   monitor='avg_val_loss',
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode='min'
)

checkpoint = ModelCheckpoint(monitor='avg_val_loss', mode='min')

In [32]:
trainer = pl.Trainer(callbacks=[early_stop_callback, checkpoint], gpus=[0], max_epochs=1, fast_dev_run=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [33]:
torch.cuda.empty_cache()
import gc

gc.collect()
torch.cuda.empty_cache()

In [34]:
trainer.fit(model, train_dataloder, val_dataloder)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                          | Params
----------------------------------------------------------
0 | seq2seq | MBartForConditionalGeneration | 70.1 M
----------------------------------------------------------
70.1 M    Trainable params
0         Non-trainable params
70.1 M    Total params
280.283   Total estimated model params size (MB)


In [35]:
def model_device(model):
    return next(iter(model.parameters())).device

In [95]:
def generate(model, input_ids, input_attention_mask, **gen_args):
    model.eval()
    device = model_device(model)
    input_ids, input_attention_mask = input_ids.to(device), input_attention_mask.to(device)
    
    
    # Да, EOS а не BOS, новая мода
    decoder_start_token = model.seq2seq.config.eos_token_id
    # приходится эмбеддить руками, чтобы в encoder шли inputs_embeds
    
    inputs_embeds = model.seq2seq.get_input_embeddings()
    with torch.no_grad():
        encoder_outputs = model.seq2seq.get_encoder()(attention_mask=input_attention_mask, input_ids=input_ids)
        
        output = model.seq2seq.generate(encoder_outputs=encoder_outputs,
                               decoder_start_token_id=decoder_start_token,
                               **gen_args)
        
        return tokenizer.decode(output[0].tolist(), skip_special_tokens=True)

In [37]:
def generate_from_text(model, text, **gen_args):
    tokenized = tokenizer(text, return_tensors='pt')
    return generate(model, tokenized['input_ids'], tokenized['attention_mask'], **gen_args)

In [38]:
tgt_texts[15000] 

NameError: ignored

In [96]:
print(generate_from_text(model, "Исправь миня", num_beams=10))

Исправь.


In [53]:
torch.save(model.state_dict(), './model')

In [54]:
model_new = BartConditionalGeneration(hparams) 
model_new.load_state_dict(torch.load('./model'))


<All keys matched successfully>

In [64]:
print(generate_from_text(model_new, "I have no idea.", num_beams=10))

</s>ru_RU IMadbody.</s>
